In [68]:
import requests
from bs4 import BeautifulSoup
import json
import pandas

In [92]:
def read_file(file_name):
    """
    读取输入内容
    :param file_name:
    :return:
    """
    train_dict = {}
    with open(file_name) as f:
        current_id = ''
        for line in f:
            if '#id' in line.split(':'):
                if current_id != line.split(':')[1]:
                    current_id = line.split(':')[1].strip()
                    train_dict[current_id] = {'name':'', 'org':'', 'search_results_page':'',
                                             'homepage':'', 'pic':'', 'email':'', 'gender':'',
                                             'position':'', 'location':''}
            if '#name' in line.split(':'):
                train_dict[current_id]['name'] = line.split(':')[1].strip()
            if '#org' in line.split(':'):
                train_dict[current_id]['org'] = line.split(':')[1].strip()
            if '#search_results_page' in line.split(':'):
                train_dict[current_id]['search_results_page'] = line.split('#search_results_page:')[1].strip()

    return train_dict

In [93]:
result = read_file('small_training.txt')

In [94]:
result

{'5616d8a645cedb3397b889d7': {'email': '',
  'gender': '',
  'homepage': '',
  'location': '',
  'name': 'Chen Zhang',
  'org': 'Peking University(Peking University),Beijing,China',
  'pic': '',
  'position': '',
  'search_results_page': 'http://ifang.ml:8081/5616d8a645cedb3397b889d7.html'}}

In [115]:
def scrap_from_google(words, file_name):
    """
    根据给定词语, 缓存 google 第一页搜索结果至指定文件
    https://www.google.com.hk/search?q=[words]
    :param words: 给定的关键词用空格分开
    :param file_name: 要存储的文件名字
    :return
    """
    url = 'https://www.google.com.hk/search?q=' + words
    response = requests.get(url)
    with open('results/'+file_name, 'w') as f:
        f.write(response.text)

In [116]:
for key in result:
    words = result[key]['name'] + ' ' + result[key]['org']
    file_name = result[key]['search_results_page'].split('/')[-1]
    scrap_from_google(words, file_name)

In [139]:
def extract_url(file_name):
    """
    将 google 搜索结果页的所有 url 解析出来
    :param file_name:
    :return:
    """
    with open('results/'+file_name) as f:
        result_page = BeautifulSoup(''.join(f.readlines()), 'lxml')
        results = result_page.find('div',{'id':'ires'})
        for url in results.find_all('a'):
            print(url.get('href'))
            print('-----')


In [140]:
for key in result:
    file_name = result[key]['search_results_page'].split('/')[-1]
    extract_url(file_name)

/url?q=http://mgv.pku.edu.cn/%3Fcatalog%3Denpiintro%26pname%3DChen_Zhang&sa=U&ved=0ahUKEwi6sMyoxoPWAhUBy7wKHUwqC8cQFggTMAA&usg=AFQjCNE7gGwLNpzPUCQ7nlZu52z8Twzc1g
-----
/url?q=http://webcache.googleusercontent.com/search%3Fq%3Dcache:8y6bZdlEK_cJ:http://mgv.pku.edu.cn/%3Fcatalog%253Denpiintro%2526pname%253DChen_Zhang%252BChen%2BZhang%2BPeking%2BUniversity(Peking%2BUniversity),Beijing,China%26newwindow%3D1%26hl%3Dzh-TW%26ct%3Dclnk&sa=U&ved=0ahUKEwi6sMyoxoPWAhUBy7wKHUwqC8cQIAgWMAA&usg=AFQjCNFER_skpHEsZNpOkMnGoGCqmeC8WQ
-----
/search?newwindow=1&ie=UTF-8&q=related:mgv.pku.edu.cn/%3Fcatalog%3Denpiintro%26pname%3DChen_Zhang+Chen+Zhang+Peking+University(Peking+University),Beijing,China&tbo=1&sa=X&ved=0ahUKEwi6sMyoxoPWAhUBy7wKHUwqC8cQHwgXMAA
-----
/url?q=https://www.researchgate.net/profile/Chen_Zhang135&sa=U&ved=0ahUKEwi6sMyoxoPWAhUBy7wKHUwqC8cQFggZMAE&usg=AFQjCNEke6qWGbWhIMvhNlgQNrmEEJa7Ww
-----
/url?q=https://www.researchgate.net/profile/Chen_Zhang82&sa=U&ved=0ahUKEwi6sMyoxoPWAhUBy7wKHUwqC8c